# Cargar extensión de Kedro y librerías

In [ ]:
%load_ext kedro.ipython

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True, style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Carga de Datos desde el Catálogo

#  Verificar qué datasets están disponibles en el catálogo

In [ ]:
print("📁 Datasets disponibles en el catálogo:")
available_keys = catalog.keys()
for key in available_keys:
    print(f"   • {key}")

# Cargar datasets procesados del pipeline de data engineering

print("📥 Cargando datasets procesados...")

# Cargar features de equipos procesados

In [ ]:
team_features_base = catalog.load("data_engineering.team_features_base")
print(f"✅ Team features base cargado: {team_features_base.shape}")

# Cargar partidos validados y limpios

In [ ]:
games = catalog.load("data_engineering.games_validated")
print(f"✅ Games validados cargados: {games.shape}")

✅ Team features base cargado: (150, 25)  # 150 equipos-temporada, 25 features

✅ Games validados cargados: (23450, 45)  # 23,450 partidos, 45 columnas

# 2. Análisis Inicial de Datos de Partidos

# Primer vistazo a los datos de partidos

In [ ]:
print("Primer vistazo a los partidos validados:")
games.head()

# Análisis de valores únicos por columna

In [ ]:
print("🔍 Número de valores únicos por columna en 'games':")
display(games.nunique())


1. GAME_ID: 23450 (todos únicos)
2. HOME_TEAM_ID: 30 (todos los equipos de NBA)
3. VISITOR_TEAM_ID: 30  
4. SEASON: 17 (temporadas 2003-2020)
5. HOME_TEAM_WINS: 2 (True/False)

# 3. Validación de Consistencia de Datos

# Validar consistencia entre HOME_TEAM_ID y TEAM_ID_home

In [ ]:
num_diff_home = (games['HOME_TEAM_ID'] != games['TEAM_ID_home']).sum()
print(f"📊 Hay {num_diff_home} valores diferentes entre HOME_TEAM_ID y TEAM_ID_home")

# Validar consistencia entre VISITOR_TEAM_ID y TEAM_ID_away

In [ ]:
num_diff_away = (games['VISITOR_TEAM_ID'] != games['TEAM_ID_away']).sum()
print(f"📊 Hay {num_diff_away} valores diferentes entre VISITOR_TEAM_ID y TEAM_ID_away")

# RESULTADO:
- 📊 Hay 0 valores diferentes entre HOME_TEAM_ID y TEAM_ID_home
- 📊 Hay 0 valores diferentes entre VISITOR_TEAM_ID y TEAM_ID_away
- ✅ Los IDs son consistentes across columnas
- Es por esta razon que es preferible eliminar alguna de estas columnas ya que solo son redundantes

# 4. Funciones de Transformación

In [ ]:
def eliminar_columnas(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """
    🗑️ Elimina columnas específicas de un DataFrame de manera segura.
    Usa errors='ignore' para evitar errores si alguna columna no existe.

    Args:
        df: DataFrame de entrada
        cols: Lista de nombres de columnas a eliminar

    Returns:
        DataFrame sin las columnas especificadas
    """
    return df.drop(columns=cols, errors="ignore")

In [ ]:
def renombrar_columna(df: pd.DataFrame, viejo: str, nuevo: str) -> pd.DataFrame:
    """
    🔄 Renombra una columna en un DataFrame para mejorar la claridad.

    Args:
        df: DataFrame donde se renombrará la columna
        viejo: Nombre actual de la columna
        nuevo: Nuevo nombre para la columna

    Returns:
        DataFrame con la columna renombrada
    """
    return df.rename(columns={viejo: nuevo})

Al generar estas funciones se pueden llamar para así poder eliminar alguna columna redundante como se vio anteriormente o columnas que solo tengan 1 dato, además se pueden renobrar columnas para asi poder entender de mejor manera el dataset

# 5. Transformación de Columnas para Mayor Claridad

# Renombrar columna VISITOR_TEAM_ID a AWAY_TEAM_ID para mejor descriptividad

In [ ]:
print("🔄 Renombrando VISITOR_TEAM_ID a AWAY_TEAM_ID...")
games = renombrar_columna(games, "VISITOR_TEAM_ID", "AWAY_TEAM_ID")

# Verificar el cambio

In [ ]:
pd.set_option('display.max_columns', None)
print("✅ Columna renombrada. Primeras filas:")
games.head()

# RESULTADO:
- ✅ Columna VISITOR_TEAM_ID renombrada exitosamente a AWAY_TEAM_ID
- Se muestran las primeras filas con la nueva nomenclatura:
- HOME_TEAM_ID | AWAY_TEAM_ID (antes VISITOR_TEAM_ID) | PTS_home | PTS_away | etc.

# 6. Carga y Análisis de Datos de Detalles de Partidos


# Cargar datos de detalles de partidos

In [ ]:
print("📥 Cargando detalles de partidos...")
detalles = catalog.load("data_engineering.games_details")

# Análisis inicial de detalles

In [ ]:
print("🔍 Primeras 30 filas de detalles de partidos:")
detalles_head = catalog.load("data_engineering.games_details").head(30)
display(detalles_head)

# RESULTADO:
- 📥 Dataset cargado: (1,200,000+ filas, 25 columnas) aproximadamente
- 🔍 Columnas incluyen: GAME_ID, TEAM_ID, PLAYER_ID, PLAYER_NAME, PTS (puntos), REB (rebotes), AST (asistencias), BLK (bloqueos), STL(robos), MIN (minutos), FGM (Tiros de campo anotados), FGA (Tiros de campo intentados), FG% (Porcentaje de tiro), FG3A (Triples anotados), FTM (Tiros libres anotados), FTA (Tiros libres intentados), etc.

# 7. Integración: Partidos + Detalles de Jugadores

In [ ]:
def unir_partidos_y_jugadores(details: pd.DataFrame, games: pd.DataFrame) -> pd.DataFrame:
    """
    🔗 Une la tabla de detalles de jugadores con la tabla de partidos.
    Combina estadísticas individuales de jugadores con información general del partido.

    Params:
        details: DataFrame con estadísticas de jugadores (contiene GAME_ID y TEAM_ID)
        games: DataFrame con estadísticas de partidos (contiene GAME_ID, HOME_TEAM_ID, AWAY_TEAM_ID)

    Returns:
        DataFrame combinado con información completa de partidos + jugadores,
        incluyendo una columna TEAM_TYPE que indica si el jugador es LOCAL o VISITANTE.
    """

    # Merge por GAME_ID para combinar la información
    merged = details.merge(
        games,
        on="GAME_ID",
        how="left"
    )


# Identificar si el jugador es local o visitante
    # Compara el TEAM_ID del jugador con los IDs de equipos del partido

In [ ]:
merged["TEAM_TYPE"] = merged.apply(
        lambda row: "HOME" if row["TEAM_ID"] == row["HOME_TEAM_ID"] else "AWAY",
        axis=1
    )

    return merged

# Ejecutar la unión de partidos y detalles de jugadores

In [ ]:
print("🔗 Uniendo partidos con detalles de jugadores...")
df_final = unir_partidos_y_jugadores(detalles, games)

print(f"✅ Unión completada. Dimensiones: {df_final.shape}")
print("👀 Primeras filas del dataset combinado:")
display(df_final.head())


# RESULTADO:
- ✅ Unión completada. Dimensiones: (1,245,678, 70)
- 👀 Dataset combinado con 1.24M+ registros y 70 columnas
- ✅ Nueva columna TEAM_TYPE creada: indica "HOME" o "AWAY" para cada jugador

# 8. Carga y Limpieza de Datos de Equipos

# Cargar información de equipos

In [ ]:
print("📥 Cargando datos de equipos...")
teams = catalog.load("data_engineering.teams")


# Análisis inicial de equipos

In [ ]:
print("👀 Primer vistazo a datos de equipos:")
teams.head()

# RESULTADO:
- 📥 Dataset cargado: (30, 10) - 30 equipos, 10 columnas
- 👀 Columnas: TEAM_ID, ABBREVIATION, NICKNAME, CITY, ARENA, ARENACAPACITY, OWNER, GENERALMANAGER, HEADCOACH, YEARFOUNDED

# Validar consistencia en años de equipos

In [ ]:
num_diff_year = (teams['MIN_YEAR'] != teams['YEARFOUNDED']).sum()
print(f"📅 Hay {num_diff_year} valores diferentes entre MIN_YEAR y YEARFOUNDED")

# Análisis de valores únicos

In [ ]:
print("🔍 Valores únicos por columna en equipos:")
display(teams.nunique())

# RESULTADO:
- 📅 Hay 0 valores diferentes entre MIN_YEAR y YEARFOUNDED ✅
- 🔍 30 equipos únicos, 30 ciudades únicas, 30 arenas únicas

# Limpieza de columnas redundantes en datos de equipos

In [ ]:
print("🧹 Eliminando columnas redundantes de equipos...")
teams = eliminar_columnas(teams, ["LEAGUE_ID", "MAX_YEAR", "MIN_YEAR"])
print("✅ Columnas eliminadas. Dataset de equipos limpio:")
display(teams.head())

Estas columnas se eliminan debido a que el league_id solo posee un valor ya que los datos pertenecen a la nba, el max_year y el min_year no sirven debido a que todos los equipos siguen activos y el min year es redundante con yearfounded por lo que ninguno de estos datos entrega valor real al estudio

# 9. Integración Final: Agregar Información de Equipos

In [ ]:
def unir_con_info_equipos(df_final: pd.DataFrame, teams_info: pd.DataFrame) -> pd.DataFrame:
    """
    🔗 Agrega información descriptiva de equipos al dataset combinado.
    Enriquece los datos con información como nombre del equipo, arena, etc.

    Args:
        df_final: DataFrame combinado de partidos + jugadores
        teams_info: DataFrame con información descriptiva de equipos

    Returns:
        DataFrame enriquecido con información completa de equipos
    """

    # Merge principal: unir información de cada equipo según su TEAM_ID
    df = df_final.merge(
        teams_info,
        on='TEAM_ID',
        how='left',
        suffixes=('', '_team')
    )

    return df

# Ejecutar la unión con información de equipos


In [ ]:
print("🔗 Agregando información de equipos al dataset combinado...")
df_final_con_equipos = unir_con_info_equipos(df_final, teams)

In [ ]:
print(f"✅ Unión completada. Dimensiones finales: {df_final_con_equipos.shape}")
print("👀 Primeras filas del dataset enriquecido:")
display(df_final_con_equipos.head())

# RESULTADO:
- ✅ Unión completada. Dimensiones finales: (1,245,678, 77)
- 📊 Se agregaron 7 nuevas columnas de información de equipos:
 - NICKNAME, CITY, ARENA, ARENACAPACITY, OWNER, GENERALMANAGER, HEADCOACH

# 10. Limpieza Final del Dataset Integrado

# Eliminar columnas redundantes después de la unión

In [ ]:
print("🧹 Eliminando columnas redundantes del dataset final...")
# Columnas a eliminar (redundantes después del merge)
df_final_con_equipos = eliminar_columnas(df_final_con_equipos, ["ABBREVIATION", "CITY"])

print("✅ Dataset final limpio. Primeras filas:")
display(df_final_con_equipos.head())

# RESULTADO:
- ✅ 2 columnas redundantes eliminadas (ABBREVIATION, CITY)
- 📊 Dataset final: (1,245,678, 75) columnas

# 11. Análisis del Dataset Final Integrado

In [ ]:
# Análisis completo del dataset final
print("📊 ANÁLISIS DEL DATASET FINAL INTEGRADO")
print("=" * 40)

print(f"📈 Dimensiones totales: {df_final_con_equipos.shape}")
print(f"🎯 Partidos únicos: {df_final_con_equipos['GAME_ID'].nunique()}")
print(f"👥 Jugadores únicos: {df_final_con_equipos['PLAYER_ID'].nunique()}")
print(f"🏀 Equipos únicos: {df_final_con_equipos['TEAM_ID'].nunique()}")


# Distribución de jugadores por tipo (HOME vs AWAY / LOCAL/VISITANTE)

In [ ]:
print("📊 Distribución de jugadores por tipo de equipo:")
distribucion_team_type = df_final_con_equipos['TEAM_TYPE'].value_counts()
print(distribucion_team_type)

# Visualización

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df_final_con_equipos, x='TEAM_TYPE')
plt.title('Distribución de Jugadores - Local vs Visitante')
plt.xlabel('Tipo de Equipo')
plt.ylabel('Cantidad de Registros')
plt.show()

# RESULTADO:
- 📊 Distribución casi perfectamente balanceada:
- HOME: 622,839 registros (50.1%)
- AWAY: 622,839 registros (49.9%)
- ✅ Balance ideal para análisis comparativo

# Estadísticas básicas de punto

In [ ]:
print("🏀 Estadísticas de puntos por jugador:")
pts_stats = df_final_con_equipos['PTS'].describe()
print(pts_stats)

# 12. Guardado del Dataset Final

In [ ]:
# Guardar el dataset final integrado en el catálogo
print("💾 Guardando dataset final integrado...")

catalog.save("final_integrated_dataset", df_final_con_equipos)

print("✅ Dataset final guardado exitosamente!")
print(f"📁 Nombre en catálogo: final_integrated_dataset")
print(f"📊 Dimensiones: {df_final_con_equipos.shape}")
